In [0]:
!nvidia-smi

Mon Dec  9 18:47:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 450kB 43.2MB/s 
     |████████████████████████████████| 3.8MB 48.1MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.2 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled googl

In [0]:
import tensorflow as tf
import numpy as np

import json

import os
#To free memory
import gc


In [0]:
print(tf.__version__)
print(tf.config.experimental.list_physical_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#tf.debugging.set_log_device_placement(True)

2.0.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [0]:
if not "annotations" in os.listdir():
    if not "annotations_trainval2014.zip" in os.listdir():
        !wget "http://images.cocodataset.org/annotations/annotations_trainval2014.zip"
    !unzip annotations_trainval2014.zip

--2019-12-09 19:06:19--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.224.120
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.224.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  29.1MB/s    in 8.9s    

2019-12-09 19:06:28 (27.2 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

Archive:  annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  


In [0]:
if not "train2014" in os.listdir():
    if not "train2014.zip" in os.listdir():
        !wget "http://images.cocodataset.org/zips/train2014.zip"
    !unzip train2014.zip

In [0]:
annotations_file = open("annotations/captions_train2014.json")
annotation_js = json.load(annotations_file)

In [0]:
annotation_list = [(j["image_id"],j["caption"]) for j in annotation_js["annotations"]]
annotation_list.sort()
annotation_list

data = [{"id" : annotation_list[0][0] , "captions" : [annotation_list[0][1]]}]

for img_id , capt in annotation_list[1:]:
    
    if data[-1]["id"] != img_id:
        data.append({"id" : img_id , "captions" : []})
        
    data[-1]["captions"].append(capt)
    
data

[{'captions': ['A bunch of trays that have different food.',
   'A meal is presented in brightly colored plastic trays.',
   'Closeup of bins of food that include broccoli and bread.',
   'Colorful dishes holding meat, vegetables, fruit, and bread.',
   'there are containers filled with different kinds of foods'],
  'id': 9},
 {'captions': ['A giraffe eating food from the top of the tree.',
   'A giraffe mother with its baby in the forest.',
   'A giraffe standing next to a forest filled with trees.',
   'A giraffe standing up nearby a tree ',
   'Two giraffes standing in a tree filled area.'],
  'id': 25},
 {'captions': ['A flower vase is sitting on a porch stand.',
   'A vase with red and white flowers outside on a sunny day.',
   'A white vase filled with different colored flowers.',
   'White vase with different colored flowers sitting inside of it. ',
   'a white vase with many flowers on a stage'],
  'id': 30},
 {'captions': ['A lone zebra grazing in some green grass.',
   'A zeb

In [0]:
annotation_list = annotation_list[:10000]

## Preprocessing

In [0]:
#To have a small set, for fast experiments
data_small = data[:100]

### Text: Captions

In [0]:
annotation_list = ["<start> " + annotation[1] + " <end>" for annotation in annotation_list]

In [0]:
annotation_list

['<start> A bunch of trays that have different food. <end>',
 '<start> A meal is presented in brightly colored plastic trays. <end>',
 '<start> Closeup of bins of food that include broccoli and bread. <end>',
 '<start> Colorful dishes holding meat, vegetables, fruit, and bread. <end>',
 '<start> there are containers filled with different kinds of foods <end>',
 '<start> A giraffe eating food from the top of the tree. <end>',
 '<start> A giraffe mother with its baby in the forest. <end>',
 '<start> A giraffe standing next to a forest filled with trees. <end>',
 '<start> A giraffe standing up nearby a tree  <end>',
 '<start> Two giraffes standing in a tree filled area. <end>',
 '<start> A flower vase is sitting on a porch stand. <end>',
 '<start> A vase with red and white flowers outside on a sunny day. <end>',
 '<start> A white vase filled with different colored flowers. <end>',
 '<start> White vase with different colored flowers sitting inside of it.  <end>',
 '<start> a white vase wit

In [0]:
annotation_lens = [ len(ann.split()) for ann in annotation_list]
max(annotation_lens)

44

In [0]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(#num_words=top_k,
    oov_token="<unk>", filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(annotation_list)

In [0]:
wc = tokenizer.word_counts.values()
mean_wc = sum(wc)/len(wc)
z_scores = [i - mean_wc for i in wc]
z_scores = []

In [0]:
embedding_count = len(tokenizer.word_index.keys())+1

In [0]:
# Recorver the word index that was created with the tokenizer
word_index = tokenizer.word_index
print('Found {} unique tokens.\n'.format(len(word_index)))
word_count = tokenizer.word_counts
print("Show the most frequent word index:")
for i, word in enumerate(sorted(word_count, key=word_count.get, reverse=True)):
    print('   {} ({}) --> {}'.format(word, word_count[word], word_index[word]))
    if i == 20: 
        print('')
        break

print("Show the least frequent word index:")
for i, word in enumerate(sorted(word_count, key=word_count.get, reverse=False)):
    print('   {} ({}) --> {}'.format(word, word_count[word], word_index[word]))
    if i == 20: 
        print('')
        break


Found 4758 unique tokens.

Show the most frequent word index:
   a (16507) --> 2
   <start> (10000) --> 3
   <end> (10000) --> 4
   on (4008) --> 5
   of (3534) --> 6
   the (3328) --> 7
   in (3030) --> 8
   with (2597) --> 9
   and (2370) --> 10
   is (1666) --> 11
   man (1145) --> 12
   to (1077) --> 13
   sitting (899) --> 14
   an (859) --> 15
   two (773) --> 16
   people (769) --> 17
   standing (698) --> 18
   at (689) --> 19
   are (662) --> 20
   white (587) --> 21
   next (552) --> 22

Show the least frequent word index:
   presented (1) --> 2692
   porch (1) --> 2693
   ruched (1) --> 2694
   costumed (1) --> 2695
   hind (1) --> 2696
   reared (1) --> 2697
   brave (1) --> 2698
   6 (1) --> 2699
   10 (1) --> 2700
   outer (1) --> 2701
   seem (1) --> 2702
   embracing (1) --> 2703
   nuzzle (1) --> 2704
   lapse (1) --> 2705
   grafitti (1) --> 2706
   decked (1) --> 2707
   engined (1) --> 2708
   polished (1) --> 2709
   exhaust (1) --> 2710
   allows (1) --> 2711
   o

In [0]:
max_window = max(annotation_lens)
capt_train = tokenizer.texts_to_sequences(annotation_list)
capt_train = tf.keras.preprocessing.sequence.pad_sequences(capt_train,maxlen=max_window)
capt_train

array([[  0,   0,   0, ..., 180,  68,   4],
       [  0,   0,   0, ..., 496, 748,   4],
       [  0,   0,   0, ...,  10, 318,   4],
       ...,
       [  0,   0,   0, ..., 108, 150,   4],
       [  0,   0,   0, ...,   7, 493,   4],
       [  0,   0,   0, ...,  15, 115,   4]], dtype=int32)

### Images

In [0]:
def load_images(ids):
    images = []
    zero_padding = 12

    for idd in ids:

        img_id = str(idd)
        pic_name = "train2014/COCO_train2014_" + ("0"*(zero_padding - len(img_id))) + img_id + ".jpg"

        img = tf.keras.preprocessing.image.load_img(pic_name, target_size=(224, 224))
        pre_img = tf.keras.preprocessing.image.img_to_array(img)
        images.append(pre_img)
        
        
        
    np_images = np.array(images)
    del(images)
    pre_np_images = tf.keras.applications.resnet_v2.preprocess_input(np_images)
    del(np_images)
    
    return pre_np_images
    

# D1: Basic caption generation model
In this section we build a basic modern caption generation model. To generate captions out of images, we embrace the encoder-decoder architecture, which consists in encoding the image in a dense short vector (with a CNN for example) and decoding the vector as a sequence of words (using some sort of RNN, e.g. LSTMs).

...

## Encoder
We are going to use a State-Of-The-Art image classification network as our image encoder. The network in question is a CNN based residual net called ResNet.

In [0]:
resnet152 = tf.keras.applications.ResNet152V2(weights="imagenet") #load ResNet152, trained on imagenet
resnet152.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [0]:
some_images = load_images([datum["id"] for datum in data_small])
pred = resnet152.predict(some_images)
pred_classes = tf.keras.applications.resnet_v2.decode_predictions(pred,top=3)
print(pred_classes)

# To free space, trust me you'll need this
%xdel pred
%xdel some_images
%xdel pred_classes

gc.collect()

[[('n07715103', 'cauliflower', 0.6370351), ('n07714990', 'broccoli', 0.35870326), ('n07711569', 'mashed_potato', 0.0042259563)], [('n02130308', 'cheetah', 0.85301125), ('n02127052', 'lynx', 0.111899145), ('n02128385', 'leopard', 0.02228276)], [('n04522168', 'vase', 0.8673538), ('n03991062', 'pot', 0.11452658), ('n03903868', 'pedestal', 0.0147273475)], [('n02391049', 'zebra', 1.0), ('n01518878', 'ostrich', 1.2668588e-10), ('n03447447', 'gondola', 1.0958564e-10)], [('n04507155', 'umbrella', 0.9989201), ('n03710721', 'maillot', 0.0008845), ('n03710637', 'maillot', 0.0001253053)], [('n02091831', 'Saluki', 0.3834884), ('n02090622', 'borzoi', 0.29409832), ('n02088094', 'Afghan_hound', 0.1393124)], [('n01580077', 'jay', 0.4168986), ('n02484975', 'guenon', 0.089360826), ('n02488291', 'langur', 0.081555486)], [('n02708093', 'analog_clock', 0.98425645), ('n04548280', 'wall_clock', 0.015669608), ('n02794156', 'barometer', 7.205418e-05)], [('n03272562', 'electric_locomotive', 0.406674), ('n0339391

9

In [0]:
#Remove top dense layer, wrap it in a model for convenience
resnet_encoder = tf.keras.Model(inputs=resnet152.input,outputs=resnet152.layers[-2].output)
resnet_encoder.trainable = False #Freeze layers
resnet_encoder.compile(loss=tf.keras.losses.CosineSimilarity()) # Needed for the freezing to make effect

#resnet_encoder.summary()

In [0]:
#Loading all the images at once and running the network on the whole thing is not feasible for most systems
#This function loads and extracts the features of the images by batches, removing images that have already been processed
def load_image_features_batched(image_batch_size = 4000):
    image_features = []

    for batch_i in range(0,len(data),image_batch_size):

        start_i = batch_i
        end_i = start_i + image_batch_size

        if (end_i > len(data)):
            end_i = len(data)

        ids = [datum["id"] for datum in data[start_i:end_i]]

        some_images = load_images(ids)
        pred = resnet_encoder.predict(some_images)
        image_features.append(pred)

        #Delete uneeded stuff
        gc.collect()
        
    image_features_array = np.concatenate(image_features) # All but last, since last shape might be different
    return image_features_array

In [0]:
if not "features" in os.listdir():
    os.mkdir("features")
    
if not "resnet_image_embdd.npz" in os.listdir("features"):
    image_features = load_image_features_batched()
    np.savez_compressed("features/resnet_image_embdd.npz",image_features)
    %xdel image_features
    gc.collect()

In [0]:
image_features_compressed = np.load("features/resnet_image_embdd.npz")

In [0]:
image_features = image_features_compressed["arr_0"]
image_features_compressed.close()
gc.collect()

68

## Decoder

In [0]:
hidden_len = 512
embedding_len = 512

dec_in_caption = tf.keras.Input(shape=(max_window))
#dec_in_img_enc = tf.keras.Input(shape=(embedding_len))
dec_in_cropped = dec_in_caption[:,:-1]
embdd = tf.keras.layers.Embedding(embedding_count,embedding_len,input_length=max_window)(dec_in_cropped)

#img_enc = tf.keras.layers.Reshape((1,dec_in_img_enc.shape[1]))(dec_in_img_enc)
#padded = tf.keras.layers.Concatenate(axis=1)([img_enc,embdd])

#hidden = [resnet_enc.output , tf.keras.layers.InputLayer(input_tensor=tf.)]

lstm  = tf.keras.layers.LSTM(units=hidden_len,return_sequences=True,unroll=False)(embdd)
#cropped = tf.keras.layers.Cropping1D(cropping=(1,0))(lstm)
dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=embedding_count,activation=tf.keras.activations.softmax))(lstm)
#dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=top_k+2, activation=tf.keras.activations.softmax))(lstm)

In [0]:
image_embedding_decoder = tf.keras.Model(inputs=[dec_in_caption],outputs=dense)
image_embedding_decoder.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 44)]              0         
_________________________________________________________________
tf_op_layer_strided_slice_4  [(None, 43)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 43, 512)           2436608   
_________________________________________________________________
lstm_3 (LSTM)                (None, 43, 512)           2099200   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 43, 4759)          2441367   
Total params: 6,977,175
Trainable params: 6,977,175
Non-trainable params: 0
_________________________________________________________________


In [0]:
def create_mask(y_true):
    start_index = word_index['<start>']
    capt_mask = tf.math.logical_not(tf.math.logical_or(
        y_true == 0, y_true == start_index))
    return capt_mask#.astype(float) tf.dtypes.cast ?

In [0]:
def masked_cropped_sparse_categorical_crossentropy(y_true, y_pred):
    y_true_cropped = y_true[:,1:]
    mask = create_mask(y_true_cropped)
    return tf.keras.losses.SparseCategoricalCrossentropy()(
        tf.keras.layers.Reshape((-1,embedding_count))(y_true_cropped), tf.keras.layers.Reshape((-1,embedding_count))(y_pred), sample_weight=mask)
    
#    return cropped_sparse_categorical_crossentropy

In [0]:
image_embedding_decoder.compile(loss=masked_cropped_sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam())

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="loss")

In [0]:
image_embedding_decoder.fit([capt_train],[capt_train], batch_size=64, epochs=20, callbacks=[early_stopping])

Train on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5714
Epoch 2/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5587
Epoch 3/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5478
Epoch 4/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5365
Epoch 5/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5265
Epoch 6/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5161
Epoch 7/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.5059
Epoch 8/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.4966
Epoch 9/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.4873
Epoch 10/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.4786
Epoch 11/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 0.4703
Epoch 1

In [0]:
image_embedding_decoder.save('model.h5')

In [0]:
image_embedding_decoder = tf.keras.models.load_model('model.h5', 
    custom_objects={'cropped_sparse_categorical_crossentropy': 
                    cropped_sparse_categorical_crossentropy})

In [0]:
tf.keras.backend.clear_session()

In [0]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [0]:
capt_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   3,   2, 178,   6, 748,
        25, 495, 180,  68,   4], dtype=int32)

In [0]:
sent_1 = image_embedding_decoder.predict(capt_train[0].astype(float).reshape(1, -1))[0]

In [0]:
capt_train[0].shape

(44,)

In [0]:
[reverse_word_map.get(position.argmax()) for position in sent_1][-10:]

['engined',
 'a',
 'man',
 'of',
 'bananas',
 'with',
 'have',
 'different',
 'food',
 '<end>']

In [0]:
for sentence in annotation_list[:5]:
    sentence = ' '.join(sentence.split()[:3])
    for _ in range(max_window):
        sent_tokenized = tokenizer.texts_to_sequences([sentence])
        sent_padded = tf.keras.preprocessing.sequence.pad_sequences(
        sent_tokenized,maxlen=max_window)
        pred = image_embedding_decoder.predict(sent_padded.astype(float))
        index = pred[0, -1].argmax()
        word = reverse_word_map.get(index)
        if word == "<end>":
            break
        sentence += " "+word
    print(sentence)

<start> A bunch man of riding a horses motorcycle
<start> A meal man of is a sitting small on on a a bed river
<start> Closeup of of a different pizza sized on waste
<start> Colorful dishes sculptures hanging and from a a plate banana
<start> there are is two sitting bananas on and the


In [0]:
pred.shape

(1, 50, 5002)

In [0]:
pred[0, -1].argmax()


12

In [0]:
reverse_word_map.get(4)

'<end>'

In [0]:
help(tf.keras.preprocessing.text.Tokenizer)

Help on class Tokenizer in module keras_preprocessing.text:

class Tokenizer(builtins.object)
 |  Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0, **kwargs)
 |  
 |  Text tokenization utility class.
 |  
 |  This class allows to vectorize a text corpus, by turning each
 |  text into either a sequence of integers (each integer being the index
 |  of a token in a dictionary) or into a vector where the coefficient
 |  for each token could be binary, based on word count, based on tf-idf...
 |  
 |  # Arguments
 |      num_words: the maximum number of words to keep, based
 |          on word frequency. Only the most common `num_words-1` words will
 |          be kept.
 |      filters: a string where each element is a character that will be
 |          filtered from the texts. The default is all punctuation, plus
 |          tabs and line breaks, minus the `'` character.
 |      lower: boolean

In [0]:
print(tokenizer.index_word)

{1: '<unk>', 2: 'a', 3: '<start>', 4: '<end>', 5: 'on', 6: 'of', 7: 'the', 8: 'in', 9: 'with', 10: 'and', 11: 'is', 12: 'man', 13: 'to', 14: 'sitting', 15: 'an', 16: 'two', 17: 'people', 18: 'standing', 19: 'at', 20: 'are', 21: 'white', 22: 'next', 23: 'woman', 24: 'street', 25: 'that', 26: 'table', 27: 'large', 28: 'person', 29: 'holding', 30: 'it', 31: 'some', 32: 'down', 33: 'top', 34: 'riding', 35: 'field', 36: 'group', 37: 'up', 38: 'plate', 39: 'small', 40: 'train', 41: 'front', 42: 'near', 43: 'tennis', 44: 'cat', 45: 'room', 46: 'black', 47: 'his', 48: 'walking', 49: 'baseball', 50: 'has', 51: 'dog', 52: 'by', 53: 'snow', 54: 'red', 55: 'water', 56: 'young', 57: 'sign', 58: 'grass', 59: 'blue', 60: 'bathroom', 61: 'green', 62: 'while', 63: 'bed', 64: 'playing', 65: 'for', 66: 'there', 67: 'building', 68: 'food', 69: 'clock', 70: 'looking', 71: 'side', 72: 'couple', 73: 'pizza', 74: 'beach', 75: 'three', 76: 'horse', 77: 'flying', 78: 'ball', 79: 'bus', 80: 'city', 81: 'parked',